##data loading

In [24]:
import pickle
import nltk
import pandas as pd
import re
import string
import advertools as adv #irsh stop words
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
%cd /content/

/content


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%cd drive/MyDrive/Colab\ Notebooks/5030/

/content/drive/MyDrive/Colab Notebooks/5030


Irish

In [ ]:
filenames = ['bible.txt', 'blogs.txt', 'legal.txt', 'news.txt', 'wiki.txt']

with open('irish_corpus_no_tweet.txt', 'w') as outfile:
    for names in filenames:
        with open(names) as infile:
            outfile.write(infile.read())
        outfile.write("\n")


In [ ]:
f = open('irish_corpus_no_tweet.txt', 'r')
text = f.read()
f.close()
text[:10]

'\nBriathra '

English

In [25]:
f = open('corpus.txt', 'r')
en_text = f.read()
f.close()
en_text[:10]

'When the s'

##data cleaning

In [26]:
import unicodedata

In [27]:
def lower_ga(word):
    vowel = 'AEIOU\u00c1\u00c9\u00cd\u00d3\u00da'
    ga_word = word
    if len(ga_word)>1:
        if (ga_word[0] in ['t','n']) and unicodedata.normalize('NFC', ga_word)[1] in vowel:
          ga_word = ga_word[0]+'-'+ga_word[1:]
    return ga_word.lower()

In [28]:
def clean_text_round1(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


In [29]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    return text

##Irish cleaning

In [ ]:
text_r1 = clean_text_round1(text)

In [ ]:
text_r2 = clean_text_round2(text_r1)

In [ ]:
text_r2[:1000]

' Briathra Amós aoire de chuid Theacóá Fís a bhí aige faoi Iosrael i laethanta Uiziá rí Iúdá agus i laethanta Iarobám mac Ióáis rí Iosrael dhá bhliain roimh an gcrith talún  Agus dúirt sé Tá an Tiarna ag búiríl ó Shíón ag labhairt os ard ó Iarúsailéim Tá féarach na naoirí ruadhóite agus mullach Chairmeil ag feo Mar seo a deir an Tiarna In éiric thrí pheaca na Damaisce in éiric a ceithre pheaca ní rachaidh mé siar ar mo dhaorbhreith De bhrí gur lasc siad Gileád le súiste iarainn teilgfidh mé tine síos ar theach Hazáéil a dhófaidh caisleáin Bhein Hadad brisfidh mé barraí gheata na Damaisce agus dítheoidh mé áitritheoirí Ghleann Áivin agus rialtóir Bhéit Eidin agus rachaidh muintir na Síre ar deoraíocht go Cír a deir an Tiarna Mar seo a deir an Tiarna In éiric thrí pheaca Ghazá In éiric a cheithre pheaca ní rachaidh mé siar ar an bpionós De bhrí gur dhíbir siad cine iomlán mar bhránna go hEadóm teilgfidh mé tine síos ar bhalla Ghazá a dhófaidh a páláis díbreoidh mé áitritheoirí Aisdeod ag

In [ ]:
#tokenization
text_r2_token = word_tokenize(text_r2)

In [ ]:
len(text_r2_token)

22381574

In [ ]:
text_lower = [lower_ga(word) for word in text_r2_token]

In [ ]:
with open('irish_token_no_tweet.pickle', 'wb') as handle:
    pickle.dump(text_lower, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
iris_with_sw = {}
for w in text_lower:
  if w not in iris_with_sw:
    iris_with_sw[w] = 1
  else:
    iris_with_sw[w] += 1

In [ ]:
#Irish unigram frequency
with open('irish_with_sw.pickle', 'wb') as handle:
    pickle.dump(iris_with_sw, handle, protocol=pickle.HIGHEST_PROTOCOL)

##English cleaning

In [48]:
en_text = en_text.lower()
en_r1 = clean_text_round1(en_text)
en_r1[:1000]

'when the shouting ended  the bill passed   to   sending it to the senate  where a similar proposal is being sponsored by sen george parkhouse of dallas  \nmost of the fire was directed by cotten against dallas and sen parkhouse  \nthe bill would increase from  to  the maximum loan the state could make to a local water project  \ncotten construed this as a veiled effort by parkhouse to help dallas and other large cities get money which cotten felt could better be spent providing water for rural texas  \nstatements by other legislators that dallas is paying for all its water program by local bonds  and that less populous places would benefit most by the pending bill  did not sway cottens attack  \nthe bills defenders were mostly smalltown legislators like j w buchanan of dumas  eligio  kika  de la garza of mission  sam f collins of newton and joe chapman of sulphur springs  \naustin  texas \n a houston teacher  now serving in the legislature  proposed thursday a law reducing the time sp

In [49]:
en_r2 = clean_text_round2(en_r1)
en_r2[:1000]

'when the shouting ended  the bill passed   to   sending it to the senate  where a similar proposal is being sponsored by sen george parkhouse of dallas   most of the fire was directed by cotten against dallas and sen parkhouse   the bill would increase from  to  the maximum loan the state could make to a local water project   cotten construed this as a veiled effort by parkhouse to help dallas and other large cities get money which cotten felt could better be spent providing water for rural texas   statements by other legislators that dallas is paying for all its water program by local bonds  and that less populous places would benefit most by the pending bill  did not sway cottens attack   the bills defenders were mostly smalltown legislators like j w buchanan of dumas  eligio  kika  de la garza of mission  sam f collins of newton and joe chapman of sulphur springs   austin  texas   a houston teacher  now serving in the legislature  proposed thursday a law reducing the time spent lea

In [50]:
en_tokens = word_tokenize(en_r2)
len(en_tokens)

1004529

In [51]:
#build dictionary with frequency
en_unigram = {}
for word in en_tokens:
    if word not in en_unigram:
        en_unigram[word] = 1
    else:
        en_unigram[word] += 1
len(en_unigram)

46223

In [53]:
with open('en_unigram.pickle', 'wb') as handle:
    pickle.dump(en_unigram, handle, protocol=pickle.HIGHEST_PROTOCOL)

##context (ngram frequency)

In [37]:
def getNgram(text_tokens, n):
  ngram = []
  for i in range(len(text_tokens)-n+1):
    ngram.append(text_tokens[i: i+n])
  return ngram
  

In [42]:
def getNgramFreq(ngram_list):
  ngram = {}
  for each in ngram_list:
    gram = ' '.join(each)
    if gram not in ngram:
      ngram[gram] = 1
    else:
      ngram[gram] += 1
  return ngram

##Irish ngram

bigram

In [ ]:
bigram_list = getNgram(text_lower,2)

In [ ]:
irish_bigram = {}
for each in bigram_list:
  bigram = each[0]+' '+each[1]
  if bigram not in irish_bigram:  
    irish_bigram[bigram] = 1
  else:
    irish_bigram[bigram] += 1

In [ ]:
with open('irish_bigram_no_tweet.pickle', 'wb') as handle:
    pickle.dump(irish_bigram, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(irish_bigram) #with tweets 5603999

3597524

In [ ]:
bidf = pd.DataFrame(irish_bigram.items(), columns=['bigram', 'frequency'], index=None)
bidf

,bigram,frequency
0,briathra amós,1
1,amós aoire,1
2,aoire de,1
3,de chuid,12268
4,chuid theacóá,1
...,...,...
5603994,john kiffmeyer,1
5603995,kiffmeyer ach,1
5603996,roimh taifeadadh,1
5603997,dara albaim,1


In [ ]:
bidf_sort = bidf.sort_values(by=['frequency'],ascending=False)

In [ ]:
bidf_sort.set_index([pd.Index(range(len(bidf_sort)))])

,bigram,frequency
0,ar an,164374
1,leis an,97238
2,a bhí,74885
3,go bhfuil,58209
4,ag an,58094
...,...,...
5603994,ndearnadh dlí,1
5603995,– european,1
5603996,communities water,1
5603997,water policy,1


trigram

In [63]:
file = open('irish_token_no_tweet.pickle','rb')
text_lower = pickle.load(file)
file.close()

In [65]:
trigram_list = getNgram(text_lower,3)

In [67]:
irish_trigram = getNgramFreq(trigram_list)

In [69]:
with open('irish_trigram.pickle', 'wb') as handle:
    pickle.dump(irish_trigram, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [70]:
len(irish_trigram)

9915325

##English ngram

English bigram

In [54]:
en2gram = getNgram(en_tokens,2)

In [55]:
en_bigram = getNgramFreq(en2gram)

In [56]:
len(en_bigram)

445460

In [58]:
with open('en_bigram.pickle', 'wb') as handle:
    pickle.dump(en_bigram, handle, protocol=pickle.HIGHEST_PROTOCOL)

English trigram

In [59]:
en3gram = getNgram(en_tokens,3)
en_trigram = getNgramFreq(en3gram)
len(en_trigram)

837269

In [61]:
with open('en_trigram.pickle', 'wb') as handle:
    pickle.dump(en_trigram, handle, protocol=pickle.HIGHEST_PROTOCOL)

##edit distance

In [ ]:
#candidates with 1,2 edit distance
def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def irish_edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz-áéíóú'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def irish_edits2(word): 
    return (e2 for e1 in irish_edits1(word) for e2 in irish_edits1(e1))


In [ ]:
'''
def editDistDP(str1, str2):
    m = len(str1)
    n = len(str2)
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j    
            elif j == 0:
                dp[i][j] = i    
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert
                                   dp[i-1][j],        # Remove
                                   dp[i-1][j-1])    # Replace
    #print(dp)                               
    return dp[m][n]
'''